<a href="https://colab.research.google.com/github/Muntasir2179/pytorch-learnig/blob/exercises/07_pytorch_experiment_tracking_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07. PyTorch Experiment Tracking Exercise Template

Welcome to the 07. PyTorch Experiment Tracking exercise template notebook.

> **Note:** There may be more than one solution to each of the exercises. This notebook only shows one possible example.

## Resources

1. These exercises/solutions are based on [section 07. PyTorch Transfer Learning](https://www.learnpytorch.io/07_pytorch_experiment_tracking/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.
2. See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/cO_r2FYcAjU).
3. See [other solutions on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/extras/solutions).

> **Note:** The first section of this notebook is dedicated to getting various helper functions and datasets used for the exercises. The exercises start at the heading "Exercise 1: ...".

### Get various imports and helper functions

In [ ]:
# Get regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4036, done.
remote: Counting objects: 100% (1224/1224), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 4036 (delta 1067), reused 1080 (delta 995), pack-reused 2812
Receiving objects: 100% (4036/4036), 651.50 MiB | 25.26 MiB/s, done.
Resolving deltas: 100% (2360/2360), done.
Updating files: 100% (248/248), done.


In [ ]:
# Make sure we have a GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

In [ ]:
import os
import zipfile

from pathlib import Path

import requests

def download_data(source: str,
                  destination: str,
                  remove_source: bool = True) -> Path:
    """Downloads a zipped dataset from source and unzips to destination.

    Args:
        source (str): A link to a zipped file containing data.
        destination (str): A target directory to unzip data to.
        remove_source (bool): Whether to remove the source after downloading and extracting.

    Returns:
        pathlib.Path to downloaded data.

    Example usage:
        download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                      destination="pizza_steak_sushi")
    """
    # Setup path to data folder
    data_path = Path("data/")
    image_path = data_path / destination

    # If the image folder doesn't exist, download it and prepare it...
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory exists, skipping download.")
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

        # Download pizza, steak, sushi data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)

        # Unzip pizza, steak, sushi data
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...")
            zip_ref.extractall(image_path)

        # Remove .zip file
        if remove_source:
            os.remove(data_path / target_file)

    return image_path

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] Did not find data/pizza_steak_sushi directory, creating one...
[INFO] Downloading pizza_steak_sushi.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip...
[INFO] Unzipping pizza_steak_sushi.zip data...


PosixPath('data/pizza_steak_sushi')

In [ ]:
from torch.utils.tensorboard import SummaryWriter
def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str=None):
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """
    from datetime import datetime
    import os

    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [ ]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.going_modular.engine import train_step, test_step

# Add writer parameter to train()
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: torch.utils.tensorboard.writer.SummaryWriter # new parameter to take in a writer
          ) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Stores metrics to specified writer log_dir if present.

    Args:
      model: A PyTorch model to be trained and tested.
      train_dataloader: A DataLoader instance for the model to be trained on.
      test_dataloader: A DataLoader instance for the model to be tested on.
      optimizer: A PyTorch optimizer to help minimize the loss function.
      loss_fn: A PyTorch loss function to calculate loss on both datasets.
      epochs: An integer indicating how many epochs to train for.
      device: A target device to compute on (e.g. "cuda" or "cpu").
      writer: A SummaryWriter() instance to log model results to.

    Returns:
      A dictionary of training and testing loss as well as training and
      testing accuracy metrics. Each metric has a value in a list for
      each epoch.
      In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]}
      For example if training for epochs=2:
              {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


        ### New: Use the writer parameter to track experiments ###
        # See if there's a writer, if so, log to it
        if writer:
            # Add results to SummaryWriter
            writer.add_scalars(main_tag="Loss",
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                               global_step=epoch)
            writer.add_scalars(main_tag="Accuracy",
                               tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc},
                               global_step=epoch)

            # Close the writer
            writer.close()
        else:
            pass
    ### End new ###

    # Return the filled results at the end of the epochs
    return results

### Download data

Using the same data from https://www.learnpytorch.io/07_pytorch_experiment_tracking/

In [ ]:
# Download 10 percent and 20 percent training data (if necessary)
data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")

data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

[INFO] data/pizza_steak_sushi directory exists, skipping download.
[INFO] Did not find data/pizza_steak_sushi_20_percent directory, creating one...
[INFO] Downloading pizza_steak_sushi_20_percent.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip...
[INFO] Unzipping pizza_steak_sushi_20_percent.zip data...


In [ ]:
# Setup training directory paths
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

# Setup testing directory paths (note: use the same test dataset for both to compare the results)
test_dir = data_10_percent_path / "test"

# Check the directories
print(f"Training directory 10%: {train_dir_10_percent}")
print(f"Training directory 20%: {train_dir_20_percent}")
print(f"Testing directory: {test_dir}")

Training directory 10%: data/pizza_steak_sushi/train
Training directory 20%: data/pizza_steak_sushi_20_percent/train
Testing directory: data/pizza_steak_sushi/test


In [ ]:
from torchvision import transforms

# Create a transform to normalize data distribution to be inline with ImageNet
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], # values per colour channel [red, green, blue]
                                 std=[0.229, 0.224, 0.225])

# Create a transform pipeline
simple_transform = transforms.Compose([transforms.Resize((224, 224)),
                                       transforms.ToTensor(), # get image values between 0 & 1
                                       normalize])

### Turn data into DataLoaders

In [ ]:
BATCH_SIZE = 32

# Create 10% training and test DataLoaders
train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_10_percent,
                                                                                          test_dir=test_dir,
                                                                                          transform=simple_transform,
                                                                                          batch_size=BATCH_SIZE)

# Create 20% training and test DataLoaders
train_dataloader_20_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_20_percent,
                                                                                          test_dir=test_dir,
                                                                                          transform=simple_transform,
                                                                                          batch_size=BATCH_SIZE)

# Find the number of samples/batches per dataloader (using the same test_dataloader for both experiments)
print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(train_dataloader_10_percent)} (all experiments will use the same test set)")
print(f"Number of classes: {len(class_names)}, class names: {class_names}")

Number of batches of size 32 in 10 percent training data: 8
Number of batches of size 32 in 20 percent training data: 15
Number of batches of size 32 in testing data: 8 (all experiments will use the same test set)
Number of classes: 3, class names: ['pizza', 'steak', 'sushi']


## Exercise 1: Pick a larger model from [`torchvision.models`](https://pytorch.org/vision/main/models.html) to add to the list of experiments (for example, EffNetB3 or higher)

* How does it perform compared to our existing models?
* **Hint:** You'll need to set up an exerpiment similar to [07. PyTorch Experiment Tracking section 7.6](https://www.learnpytorch.io/07_pytorch_experiment_tracking/#76-create-experiments-and-set-up-training-code).

In [ ]:
# disabling the hash code checking while loading the pretrained models by overriding the get_state_dict() funciton
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

In [ ]:
# function for freezing the base layers
def freez_base_layers(model: torch.nn.Module,
                      num_classes: int):
  # freezing the layers
  for param in model.parameters():
    param.requires_grad = False

  # confugure the output layer
  set_seeds()
  feature_numbers = model.classifier[1].in_features
  model.classifier = nn.Sequential(model.classifier[0],  # keeping the dropout layer same as before
                                   nn.Linear(in_features=feature_numbers, out_features=num_classes, bias=True))
  # making the last layer trainable
  model.classifier.requires_grad_(requires_grad=True)
  return model

# function for loading effnet_b1
def get_effnet_b1(num_classes):
  weights = torchvision.models.EfficientNet_B1_Weights.DEFAULT
  model = torchvision.models.efficientnet_b1(weights=weights)
  return freez_base_layers(model, num_classes).to(device)

# function for loading effnet_b3
def get_effnet_b3(num_classes):
  weights = torchvision.models.EfficientNet_B3_Weights.DEFAULT
  model = torchvision.models.efficientnet_b3(weights=weights)
  return freez_base_layers(model, num_classes).to(device)

In [ ]:
# create epoch list
num_epochs = [5, 10]

# create models list (need to create a new model for each experiment)
models = ["effnetb1", "effnetb3"]

# create a dataloaders dictionary
train_dataloaders = {
    "data_10_percent": train_dataloader_10_percent,
    "data_20_percent": train_dataloader_20_percent
}

In [ ]:
%%time
from going_modular.going_modular.utils import save_model

set_seeds()
experiment_number = 0

# loop through each dataloader
for dataloader_name, train_dataloader in train_dataloaders.items():
  # loop through each epoch
  for epochs in num_epochs:
    # loop through each model
    for model_name in models:
      # print out info
      experiment_number += 1
      print(f"[INFO] Experiment number: {experiment_number}")
      print(f"[INFO] Model: {model_name}")
      print(f"[INFO] DataLoader: {dataloader_name}")
      print(f"[INFO] Number of epochs: {epochs}")

      # select and create the model
      if model_name == "effnetb1":
        model = get_effnet_b1(len(class_names))
      else:
        model = get_effnet_b3(len(class_names))

      # loss function and optimizer for the training
      loss_fn = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

      # train the model
      train(model=model,
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            optimizer=optimizer,
            loss_fn=loss_fn,
            epochs=epochs,
            device=device,
            writer=create_writer(experiment_name=dataloader_name,
                                 model_name=model_name,
                                 extra=f"{epochs}_epochs"))

      # save the model to file so that we can use it later if need be
      save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
      save_model(model=model,
                 target_dir="models",
                 model_name=save_filepath)
      print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model: effnetb1
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5


Downloading: "https://download.pytorch.org/models/efficientnet_b1-c27df63c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1-c27df63c.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 186MB/s]


[INFO] Created SummaryWriter, saving to: runs/2023-12-25/data_10_percent/effnetb1/5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0543 | train_acc: 0.5625 | test_loss: 0.9791 | test_acc: 0.6932
Epoch: 2 | train_loss: 0.9662 | train_acc: 0.7031 | test_loss: 0.9293 | test_acc: 0.6629
Epoch: 3 | train_loss: 0.9257 | train_acc: 0.5312 | test_loss: 0.8594 | test_acc: 0.7652
Epoch: 4 | train_loss: 0.8830 | train_acc: 0.7773 | test_loss: 0.7716 | test_acc: 0.9384
Epoch: 5 | train_loss: 0.7694 | train_acc: 0.8984 | test_loss: 0.7455 | test_acc: 0.9176
[INFO] Saving model to: models/07_effnetb1_data_10_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 2
[INFO] Model: effnetb3
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5


...

Epoch: 1 | train_loss: 0.9879 | train_acc: 0.5521 | test_loss: 0.8629 | test_acc: 0.8551
Epoch: 2 | train_loss: 0.7596 | train_acc: 0.7917 | test_loss: 0.6647 | test_acc: 0.8759
Epoch: 3 | train_loss: 0.5731 | train_acc: 0.8979 | test_loss: 0.5749 | test_acc: 0.8864
Epoch: 4 | train_loss: 0.5194 | train_acc: 0.8521 | test_loss: 0.5348 | test_acc: 0.8456
Epoch: 5 | train_loss: 0.4518 | train_acc: 0.8958 | test_loss: 0.4723 | test_acc: 0.8759
[INFO] Saving model to: models/07_effnetb3_data_20_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 7
[INFO] Model: effnetb1
[INFO] DataLoader: data_20_percent
[INFO] Number of epochs: 10
[INFO] Created SummaryWriter, saving to: runs/2023-12-25/data_20_percent/effnetb1/10_epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0038 | train_acc: 0.6042 | test_loss: 0.8834 | test_acc: 0.8873
Epoch: 2 | train_loss: 0.8128 | train_acc: 0.8479 | test_loss: 0.7442 | test_acc: 0.8570
Epoch: 3 | train_loss: 0.6998 | train_acc: 0.8417 | test_loss: 0.6073 | test_acc: 0.9489
Epoch: 4 | train_loss: 0.6058 | train_acc: 0.8667 | test_loss: 0.5542 | test_acc: 0.9280
Epoch: 5 | train_loss: 0.5493 | train_acc: 0.8771 | test_loss: 0.4987 | test_acc: 0.9280
Epoch: 6 | train_loss: 0.4679 | train_acc: 0.9292 | test_loss: 0.4343 | test_acc: 0.9384
Epoch: 7 | train_loss: 0.4480 | train_acc: 0.9167 | test_loss: 0.4047 | test_acc: 0.9280
Epoch: 8 | train_loss: 0.4589 | train_acc: 0.8875 | test_loss: 0.4458 | test_acc: 0.9072
Epoch: 9 | train_loss: 0.3866 | train_acc: 0.9167 | test_loss: 0.4165 | test_acc: 0.8977
Epoch: 10 | train_loss: 0.4143 | train_acc: 0.8896 | test_loss: 0.3480 | test_acc: 0.9384
[INFO] Saving model to: models/07_effnetb1_data_20_percent_10_epochs.pth
------------------------------------

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9879 | train_acc: 0.5521 | test_loss: 0.8629 | test_acc: 0.8551
Epoch: 2 | train_loss: 0.7596 | train_acc: 0.7917 | test_loss: 0.6647 | test_acc: 0.8759
Epoch: 3 | train_loss: 0.5731 | train_acc: 0.8979 | test_loss: 0.5749 | test_acc: 0.8864
Epoch: 4 | train_loss: 0.5194 | train_acc: 0.8521 | test_loss: 0.5348 | test_acc: 0.8456
Epoch: 5 | train_loss: 0.4518 | train_acc: 0.8958 | test_loss: 0.4723 | test_acc: 0.8759
Epoch: 6 | train_loss: 0.4307 | train_acc: 0.8771 | test_loss: 0.4815 | test_acc: 0.8049
Epoch: 7 | train_loss: 0.3808 | train_acc: 0.9146 | test_loss: 0.4355 | test_acc: 0.8551
Epoch: 8 | train_loss: 0.4009 | train_acc: 0.8708 | test_loss: 0.4531 | test_acc: 0.8049
Epoch: 9 | train_loss: 0.3618 | train_acc: 0.9104 | test_loss: 0.3990 | test_acc: 0.8561
Epoch: 10 | train_loss: 0.3436 | train_acc: 0.9167 | test_loss: 0.4231 | test_acc: 0.8049
[INFO] Saving model to: models/07_effnetb3_data_20_percent_10_epochs.pth
------------------------------------

## Exercise 2. Introduce data augmentation to the list of experiments using the 20% pizza, steak, sushi training and test datasets, does this change anything?
    
* For example, you could have one training DataLoader that uses data augmentation (e.g. `train_dataloader_20_percent_aug` and `train_dataloader_20_percent_no_aug`) and then compare the results of two of the same model types training on these two DataLoaders.
* **Note:** You may need to alter the `create_dataloaders()` function to be able to take a transform for the training data and the testing data (because you don't need to perform data augmentation on the test data). See [04. PyTorch Custom Datasets section 6](https://www.learnpytorch.io/04_pytorch_custom_datasets/#6-other-forms-of-transforms-data-augmentation) for examples of using data augmentation or the script below for an example:

```python
# Note: Data augmentation transform like this should only be performed on training data
train_transform_data_aug = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    normalize
])

# Create a helper function to visualize different augmented (and not augmented) images
def view_dataloader_images(dataloader, n=10):
    if n > 10:
        print(f"Having n higher than 10 will create messy plots, lowering to 10.")
        n = 10
    imgs, labels = next(iter(dataloader))
    plt.figure(figsize=(16, 8))
    for i in range(n):
        # Min max scale the image for display purposes
        targ_image = imgs[i]
        sample_min, sample_max = targ_image.min(), targ_image.max()
        sample_scaled = (targ_image - sample_min)/(sample_max - sample_min)

        # Plot images with appropriate axes information
        plt.subplot(1, 10, i+1)
        plt.imshow(sample_scaled.permute(1, 2, 0)) # resize for Matplotlib requirements
        plt.title(class_names[labels[i]])
        plt.axis(False)

# Have to update `create_dataloaders()` to handle different augmentations
import os
from torch.utils.data import DataLoader
from torchvision import datasets

NUM_WORKERS = os.cpu_count() # use maximum number of CPUs for workers to load data

# Note: this is an update version of data_setup.create_dataloaders to handle
# differnt train and test transforms.
def create_dataloaders(
    train_dir,
    test_dir,
    train_transform, # add parameter for train transform (transforms on train dataset)
    test_transform,  # add parameter for test transform (transforms on test dataset)
    batch_size=32, num_workers=NUM_WORKERS
):
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=train_transform)
    test_data = datasets.ImageFolder(test_dir, transform=test_transform)

    # Get class names
    class_names = train_data.classes

    # Turn images into data loaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names
```

In [ ]:
train_transform_data_aug = transforms.Compose([transforms.Resize((224, 224)),
                                               transforms.TrivialAugmentWide(),
                                               transforms.ToTensor(),
                                               normalize])

# Create 20% training and test DataLoaders
train_dataloader_20_percent_eug, test_dataloader_aug, class_names = data_setup.create_dataloaders(train_dir=train_dir_20_percent,
                                                                                              test_dir=test_dir,
                                                                                              transform=simple_transform,
                                                                                              batch_size=BATCH_SIZE)
train_dataloader_20_percent_eug, test_dataloader_aug, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7f5528412da0>,
 ['pizza', 'steak', 'sushi'])

In [ ]:
train_dataloaders["data_20_percent_aug"] = train_dataloader_20_percent_eug

In [ ]:
train_dataloaders

{'data_10_percent': <torch.utils.data.dataloader.DataLoader at 0x7f5528413310>,
 'data_20_percent': <torch.utils.data.dataloader.DataLoader at 0x7f53f640c970>,
 'data_20_percent_aug': <torch.utils.data.dataloader.DataLoader at 0x7f5528412da0>}

In [ ]:
%%time
from going_modular.going_modular.utils import save_model

set_seeds()
experiment_number = 0

# loop through each dataloader
for dataloader_name, train_dataloader in train_dataloaders.items():
  # loop through each epoch
  for epochs in num_epochs:
    # loop through each model
    for model_name in models:
      # print out info
      experiment_number += 1
      print(f"[INFO] Experiment number: {experiment_number}")
      print(f"[INFO] Model: {model_name}")
      print(f"[INFO] DataLoader: {dataloader_name}")
      print(f"[INFO] Number of epochs: {epochs}")

      # select and create the model
      if model_name == "effnetb1":
        model = get_effnet_b1(len(class_names))
      else:
        model = get_effnet_b3(len(class_names))

      # loss function and optimizer for the training
      loss_fn = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

      # train the model
      train(model=model,
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            optimizer=optimizer,
            loss_fn=loss_fn,
            epochs=epochs,
            device=device,
            writer=create_writer(experiment_name=dataloader_name,
                                 model_name=model_name,
                                 extra=f"{epochs}_epochs"))

      # save the model to file so that we can use it later if need be
      save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
      save_model(model=model,
                 target_dir="models",
                 model_name=save_filepath)
      print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model: effnetb1
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Created SummaryWriter, saving to: runs/2023-12-25/data_10_percent/effnetb1/5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0543 | train_acc: 0.5625 | test_loss: 0.9791 | test_acc: 0.6932
Epoch: 2 | train_loss: 0.9662 | train_acc: 0.7031 | test_loss: 0.9293 | test_acc: 0.6629
Epoch: 3 | train_loss: 0.9257 | train_acc: 0.5312 | test_loss: 0.8594 | test_acc: 0.7652
Epoch: 4 | train_loss: 0.8830 | train_acc: 0.7773 | test_loss: 0.7716 | test_acc: 0.9384
Epoch: 5 | train_loss: 0.7694 | train_acc: 0.8984 | test_loss: 0.7455 | test_acc: 0.9176
[INFO] Saving model to: models/07_effnetb1_data_10_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 2
[INFO] Model: effnetb3
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 5
[INFO] Created SummaryWriter, saving to: runs/2023-12-25/data_10_percent/effnetb3/5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0624 | train_acc: 0.4375 | test_loss: 1.0278 | test_acc: 0.6089
Epoch: 2 | train_loss: 0.9352 | train_acc: 0.6641 | test_loss: 0.9276 | test_acc: 0.6932
Epoch: 3 | train_loss: 0.8120 | train_acc: 0.8203 | test_loss: 0.7691 | test_acc: 0.8750
Epoch: 4 | train_loss: 0.7021 | train_acc: 0.8086 | test_loss: 0.7001 | test_acc: 0.9062
Epoch: 5 | train_loss: 0.6689 | train_acc: 0.8242 | test_loss: 0.6527 | test_acc: 0.9062
[INFO] Saving model to: models/07_effnetb3_data_10_percent_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 3
[INFO] Model: effnetb1
[INFO] DataLoader: data_10_percent
[INFO] Number of epochs: 10
[INFO] Created SummaryWriter, saving to: runs/2023-12-25/data_10_percent/effnetb1/10_epochs...


...

Epoch: 1 | train_loss: 0.9879 | train_acc: 0.5521 | test_loss: 0.8629 | test_acc: 0.8551
Epoch: 2 | train_loss: 0.7596 | train_acc: 0.7917 | test_loss: 0.6647 | test_acc: 0.8759
Epoch: 3 | train_loss: 0.5731 | train_acc: 0.8979 | test_loss: 0.5749 | test_acc: 0.8864
Epoch: 4 | train_loss: 0.5194 | train_acc: 0.8521 | test_loss: 0.5348 | test_acc: 0.8456
Epoch: 5 | train_loss: 0.4518 | train_acc: 0.8958 | test_loss: 0.4723 | test_acc: 0.8759
[INFO] Saving model to: models/07_effnetb3_data_20_percent_aug_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 11
[INFO] Model: effnetb1
[INFO] DataLoader: data_20_percent_aug
[INFO] Number of epochs: 10
[INFO] Created SummaryWriter, saving to: runs/2023-12-25/data_20_percent_aug/effnetb1/10_epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0038 | train_acc: 0.6042 | test_loss: 0.8834 | test_acc: 0.8873
Epoch: 2 | train_loss: 0.8128 | train_acc: 0.8479 | test_loss: 0.7442 | test_acc: 0.8570
Epoch: 3 | train_loss: 0.6998 | train_acc: 0.8417 | test_loss: 0.6073 | test_acc: 0.9489
Epoch: 4 | train_loss: 0.6058 | train_acc: 0.8667 | test_loss: 0.5542 | test_acc: 0.9280
Epoch: 5 | train_loss: 0.5493 | train_acc: 0.8771 | test_loss: 0.4987 | test_acc: 0.9280
Epoch: 6 | train_loss: 0.4679 | train_acc: 0.9292 | test_loss: 0.4343 | test_acc: 0.9384
Epoch: 7 | train_loss: 0.4480 | train_acc: 0.9167 | test_loss: 0.4047 | test_acc: 0.9280
Epoch: 8 | train_loss: 0.4589 | train_acc: 0.8875 | test_loss: 0.4458 | test_acc: 0.9072
Epoch: 9 | train_loss: 0.3866 | train_acc: 0.9167 | test_loss: 0.4165 | test_acc: 0.8977
Epoch: 10 | train_loss: 0.4143 | train_acc: 0.8896 | test_loss: 0.3480 | test_acc: 0.9384
[INFO] Saving model to: models/07_effnetb1_data_20_percent_aug_10_epochs.pth
--------------------------------

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9879 | train_acc: 0.5521 | test_loss: 0.8629 | test_acc: 0.8551
Epoch: 2 | train_loss: 0.7596 | train_acc: 0.7917 | test_loss: 0.6647 | test_acc: 0.8759
Epoch: 3 | train_loss: 0.5731 | train_acc: 0.8979 | test_loss: 0.5749 | test_acc: 0.8864
Epoch: 4 | train_loss: 0.5194 | train_acc: 0.8521 | test_loss: 0.5348 | test_acc: 0.8456
Epoch: 5 | train_loss: 0.4518 | train_acc: 0.8958 | test_loss: 0.4723 | test_acc: 0.8759
Epoch: 6 | train_loss: 0.4307 | train_acc: 0.8771 | test_loss: 0.4815 | test_acc: 0.8049
Epoch: 7 | train_loss: 0.3808 | train_acc: 0.9146 | test_loss: 0.4355 | test_acc: 0.8551
Epoch: 8 | train_loss: 0.4009 | train_acc: 0.8708 | test_loss: 0.4531 | test_acc: 0.8049
Epoch: 9 | train_loss: 0.3618 | train_acc: 0.9104 | test_loss: 0.3990 | test_acc: 0.8561
Epoch: 10 | train_loss: 0.3436 | train_acc: 0.9167 | test_loss: 0.4231 | test_acc: 0.8049
[INFO] Saving model to: models/07_effnetb3_data_20_percent_aug_10_epochs.pth
--------------------------------

## Exercise 3. Scale up the dataset to turn FoodVision Mini into FoodVision Big using the entire [Food101 dataset from `torchvision.models`](https://pytorch.org/vision/stable/generated/torchvision.datasets.Food101.html#torchvision.datasets.Food101)
    
* You could take the best performing model from your various experiments or even the EffNetB2 feature extractor we created in this notebook and see how it goes fitting for 5 epochs on all of Food101.
* If you try more than one model, it would be good to have the model's results tracked.
* If you load the Food101 dataset from `torchvision.models`, you'll have to create PyTorch DataLoaders to use it in training.
* **Note:** Due to the larger amount of data in Food101 compared to our pizza, steak, sushi dataset, this model will take longer to train.

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# creating a transform
data_transforms = transforms.Compose([transforms.Resize(size=(224, 224)),
                                      transforms.ToTensor(),
                                      normalize])

# get the dataset
train_data = torchvision.datasets.Food101(root="data",
                                          split="train",
                                          transform=data_transforms,
                                          download=True)

test_data = torchvision.datasets.Food101(root="data",
                                         split="test",
                                         transform=data_transforms,
                                         download=True)

100%|██████████| 4996278331/4996278331 [04:33<00:00, 18287003.69it/s]


Extracting data/food-101.tar.gz to data


In [ ]:
train_data, test_data

(Dataset Food101
     Number of datapoints: 75750
     Root location: data
     split=train
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 Dataset Food101
     Number of datapoints: 25250
     Root location: data
     split=test
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ))

In [ ]:
import os

NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 32

train_dataloader_food101 = torch.utils.data.DataLoader(dataset=train_data,
                                                       batch_size=BATCH_SIZE,
                                                       num_workers=NUM_WORKERS,
                                                       shuffle=True)

test_dataloader_food101 = torch.utils.data.DataLoader(dataset=test_data,
                                                      batch_size=BATCH_SIZE,
                                                      num_workers=NUM_WORKERS,
                                                      shuffle=False)

In [ ]:
len(train_dataloader_food101), len(test_dataloader_food101)

(2368, 790)

In [ ]:
effnetb1_food101 = get_effnet_b1(num_classes=101)
summary(model=effnetb1_food101,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=17,
        row_settings=["var_names"])

Downloading: "https://download.pytorch.org/models/efficientnet_b1-c27df63c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1-c27df63c.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 98.2MB/s]


Layer (type (var_name))                                      Input Shape       Output Shape      Param #           Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]  [1, 101]          --                Partial
├─Sequential (features)                                      [1, 3, 224, 224]  [1, 1280, 7, 7]   --                False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]  [1, 32, 112, 112] --                False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]  [1, 32, 112, 112] (864)             False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112] [1, 32, 112, 112] (64)              False
│    │    └─SiLU (2)                                         [1, 32, 112, 112] [1, 32, 112, 112] --                --
│    └─Sequential (1)                                        [1, 32, 112, 112] [1, 16, 112, 112] --                False
│    │    └─MBConv (0)       

In [ ]:
%%time
from going_modular.going_modular import engine

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=effnetb1_food101.parameters(), lr=0.001)

effnetb1_food101_rsults = engine.train(model=effnetb1_food101,
                                       train_dataloader=train_dataloader_food101,
                                       test_dataloader=test_dataloader_food101,
                                       optimizer=optimizer,
                                       loss_fn=loss_fn,
                                       epochs=5,
                                       device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 2.7403 | train_acc: 0.4065 | test_loss: 1.7636 | test_acc: 0.5802
Epoch: 2 | train_loss: 2.0415 | train_acc: 0.5098 | test_loss: 1.5134 | test_acc: 0.6159
Epoch: 3 | train_loss: 1.9057 | train_acc: 0.5348 | test_loss: 1.4475 | test_acc: 0.6298
Epoch: 4 | train_loss: 1.8372 | train_acc: 0.5474 | test_loss: 1.4044 | test_acc: 0.6399
Epoch: 5 | train_loss: 1.7889 | train_acc: 0.5566 | test_loss: 1.3925 | test_acc: 0.6351
CPU times: user 12min 33s, sys: 2min 10s, total: 14min 44s
Wall time: 44min 36s
